# <center> Optimization of Hyperparameters in a Convolution Neural Network </center>

<center>By Cecilie Dura André</center>

<img src="./Documentation/images/renderplot.png" />
Image from: https://ax.dev/tutorials/tune_cnn.html

---
# Authors

Cecilie Dura André <br>
Danish Technical University, Healthcare Technology

---
# Abstract

In the last couple of years, there has been a major shift in training and using convolutional neural networks as a second opinion in medical detection and diagnostic. It has been proven in some medical cases that the neural networks outperform the detectors [1–7]. Thus, they are thought to be able to work as a second opinion to minimize the time used on detecting and diagnosing a patient, while increasing sensitivity and specificity. Convolution neural networks can be trained by fitting the network of weights iteratively to a wished outcome by a known input [8]. For each convolution neural networks, a few hyperparameters have to be picked [8]. These parameters are often chosen before training and they are not changed under training. These parameters are picked based on experience and retraining the model a couple of times. This project will look into finding the best hyperparameters using Bayesian optimization. Thus, retraining the model is no longer necessary and time will be saved. There will also be statistical evidence for the chosen hyperparameters. 

----
# Statement of Need

The author is from a country where all medical data is saved on the same database. Data from people have been stored on this database since the end of the 1960s, thus a huge amount of data has been stored. With the right permissions, this data can be used for scientific purposes that will help doctors to see a correlation between biomarkers and diseases and to diagnose patients. This author is in a group that workes on creating algorithms that can help doctors diagnose patients as a second opinion. This software is needed because of the huge amount of data used to train and test these models. Training a model can take up to a week and if they have to be retrained multiple times months can go by. This software will make retraining unnecessary and safe weeks of training. There will also be statistical evidence for the chosen hyperparameters. 

----
# Installation instructions

Start by downloading the project by opening the terminal and write: 
    - git clone https://gitlab.msu.edu/andrecec/cmse802_spring2020_hyperparamterop.git
When the folder is cloned, the packages used to run the porject has to be installed. The folling us used for that. Python version 3.7 is used in this project and should be installed beforehand.


Open the terminal and then run:
    - conda install pytorch torchvision cpuonly -c pytorch
    - conda install numpy==1.16.1
    - conda install matplotlib==3.1.0
    - conda install botorch -c pytorch -c gpytorch
    - pip install ax-platform

Or 

Open the terminal and make sure you are in the 
"cmse802_spring2020_hyperparamterop"- folder. <br>
Then run:  <br>
    - conda env create --prefix ./envs --file ./Software/requirements.yml
    - conda activate ./envs

Or <br>

Open the terminal and make sure you are in the 
"cmse802_spring2020_hyperparamterop"- folder. 
Then run:  
    - make init  
    - conda activate ./envs 


----
# Unit Tests

When the repository is downloaded and the Python packages is installed a unit test should be made to make sure the functions works on your computer. This is done in following way: 
    - Open the terminal and make sure you are in the "cmse802_spring2020_hyperparamterop"- folder. 
    - Run "make test" in the terminal

----
# Example of implementing bayesian optimization

Other people can use this project as an example of how to implement bayesian optimization into their convolutional neural network or neural networks. To start with they will be able to see that the first difference they should make is in the training function. 

In [ ]:

def train_bayesian_optimization():
    # ...
    # Define the hyperparameters
    optimizer = optim.Adam(net.parameters(), lr=parameters.get("lr", 0.001))
    scheduler = optim.lr_scheduler.StepLR(optimizer,\
                step_size=int(parameters.get("step_size", 20)),\
                gamma=parameters.get("gamma", 1.0),)
    # ...
    for _ in range(num_epochs):
        for i in range(num_batches):
            #...
            scheduler.step()
            #...
    #...
    return net, mean_cost, accuracy

def eval_bayesian_optimization():
    #...
    # Calculating the accuracy
    return float(correct/num_batches)

In the function "train_bayesian_optimization" it can be seen that the parameters have to be given as a class. We will come back to that later in this section. The "scheduler" is then defined and it is used to affect the learning rate by decaying the learning rate of each parameter by gamma every epoch. It should be noticed that the "scheduler.step()" should be placed after the "optimizer.step()". It is important that the function gives the trained network back because it is used in Bayesian optimization. The mean cost and accuracy will be used later on for training the network and will be ignored under the bayesian optimization step. In this case,  the evaluation function, "eval_bayesian_optimization", is often the same. It should only give the accuracy back.  



In [ ]:
def evaluate_hyperparameters(parameterization):
    """ Train and evaluate the network to find the best parameters
    Args:
        parameterization: The hyperparameters that should be evaluated
    Returns:
        float: classification accuracy """
    net = Net()
    net, _, _ = train_bayesian_optimization(net=net, input_picture=DATA['x_train'],\
            label_picture=DATA['y_train'], parameters=parameterization,)

    return eval_bayesian_optimization(net=net, input_picture=DATA['x_valid'],\
            label_picture=DATA['y_valid'],)

An evaluation function has to be made. In this project it is called "evaluate_hyperparameters()", which take the class, parameters, and train a new network each time with the function "train_bayesian_optimization()" and evaluate the trained network with "eval_bayesian_optimization()" to get the accuracy.

In [2]:
from ax.service.managed_loop import optimize

run = False
if run == True:
    
    ####################################
    # THIS FOLLOWING STEP SHOULD BE USED
    ####################################
    
    BEST_PARAMETERS, VALUES, EXPERIMENT, MODEL = optimize(parameters=[{"name": "lr", "type": "range",\
        "bounds": [1e-6, 0.4], "log_scale": True},], evaluation_function=evaluate_hyperparameters,\
        objective_name='accuracy',)
    
    # Findin the best hyperparameter for training the network
    DATA1 = EXPERIMENT.fetch_data()
    DF = DATA1.df
    BEST_ARM_NAME = DF.arm_name[DF['mean'] == DF['mean'].max()].values[0]
    BEST_ARM = EXPERIMENT.arms_by_name[BEST_ARM_NAME]

Here, the class, parameters, is defined. In this project, only the best learning rate is found, but other parameters could also be used e.g. momentum, beta values, and the number of epochs. The evaluation function we defined before should be given to the "optimizer function", which is the Bayesian optimization algorithm. The next lines are used to get the best hyperparameters, which can be used to train the network.

---
# Methodology

<table>
<tr>
<td> <img src="./Documentation/images/training.png" alt="Drawing" style="width: 250px;"/> </td>
<td> <img src="./Documentation/images/costs.png" style="width: 250px;"/> </td>
</tr>
</table>
<table>
<tr>
<td> <img src="./Documentation/images/validation.png" alt="Drawing" style="width: 250px;"/> </td>
<td> <img src="./Documentation/images/test.png" style="width: 250px;"/> </td>
</tr>
</table>

Here it can be seen for both train -, validation -, and test results that the convolution neural network with hyperparameter optimization gets better results faster, but after 10 epochs the hyperparameter optimization (HO) convolution neural network and Non-HO convolution neural network looks similar in mean accuracy. The mean accuracy is the solid line and the standard deviation is the shadow with the belonging color to the line. The accuracy of the convolution neural network does not get worse with HO. This could be a concern since HO could over train the network, but are not seen here. 

We have to remember that the convolutional neural network used in this paper is small, thus there is a limit to its accuracy. HO does show promising results since the same results are achieved with fewer epochs than the convolutional neural network wit non-HO. Therefore, with bigger networks and with more hyperparameters, time might be saved in the length and with equal or better results. Now, there will also with statistical evidence for the chosen hyperparameters.

---
# Concluding Remarks

In this project, the author has learned to use Bayesian optimization with complex algorithms such as convolutional neural networks. This has contributed to me looking more into the descriptions of the functions and how they should be used. In this project, it is important since bayesian optimization and optimization of the network should not interfere with each other. The goal of this project was reached with optimal results, which indicate that further work with decisions of hyperparameters can be exchanged with this new knowledge of Bayesian optimization and knowledge of the implementation. The results from this project are also going to be shared with the research group, so they no longer have to by brute force find the optimal hyperparameters. 

No less important did the author get a sense of what it takes to write better code and write it more beautiful. Together with new methods to check whether the codes work, make environments, and in general a small insight into what it takes to write code in general. This was a pleasure. 

----
# References


[1] Sindhu Ramachandran S, Jose George, and Shibon Skaria. “Using YOLO baseddeep learning network for real time detection and localization of lung nodulesfrom low dose CT scans”. In: February 2018 (2019).doi:10.1117/12.2293699.[14]Aiden Nibali, Zhen He, and Dennis Wollersheim. “Pulmonary nodule classifica-tion with deep residual networks”. eng. In:International Journal of ComputerAssisted Radiology and Surgery12.10 (2017), pages 1799–1808.issn: 1861-6410.<br/>
[2] Wentao Zhu et al. “DeepLung: Deep 3D dual path nets for automated pul-monary nodule detection and classification”. In:Proceedings - 2018 IEEE Win-ter Conference on Applications of Computer Vision, WACV 20182018-Janua(2018), pages 673–681.doi:10.1109/WACV.2018.00079.<br/>
[3] Manu Sharma, Jignesh S Bhatt, and Manjunath V Joshi. “Early detection oflung cancer from classification using deep learning”. In: April 2018 (2019).doi:10.1117/12.2309530.<br/>
[4] Emre Dandil et al. “Artificial neural network-based classification system forlung nodules on computed tomography scans”. eng. In:2014 6th InternationalConference of Soft Computing and Pattern Recognition (SoCPaR). IEEE, 2014,pages 382–386.isbn: 9781479959341.<br/>
[5] Jinsa Kuruvilla and K Gunavathi. “Lung cancer classification using neural net-works for CT images.” eng. In:Computer methods and programs in biomedicine113.1 (2014), pages 202–209.issn: 1872-7565.url:http://search.proquest.com/docview/1461341321/.<br/>
[6] Carmen Krewer et al. “Immediate effectiveness of single-session therapeutic in-terventions in pusher behaviour.” eng. In:Gait posture37.2 (2013), pages 246–250.issn: 1879-2219.url:http://search.proquest.com/docview/1282049046/.<br/>
[7] L.B. Nascimento, A.C. De Paiva, and A.C. Silva. “Lung nodules classificationin CT images using Shannon and Simpson Diversity Indices and SVM”. In:volume 7376. 2012, pages 454–466.isbn: 9783642315367.<br/>
[8] Hargrave, Marschall. 2019. “Deep Learning.” April 30. https://www.investopedia.com/terms/d/deep-learning.asp.<br/>
[9] Aravikumar, Meghan. 2018. "Let’s Talk Bayesian Optimization." November 16. https://mlconf.com/blog/lets-talk-bayesian-optimization/.